# This notebook perform fault slip analysis (FSA) based on principal stresses from CMG geomechanical simulations.

# 1. Extract grid coordinates and fault id from Petrel exported files. Only need to do once for a 3D grid in Petrel. Save it as numpy array for later use.

# 2. Extract principla stresses (STRESMXP, STRESMNP, STRESINT) from CMG simulation results (gmch.sr3 -> rwo -> numpy).

# 3. Peform fault slip analysis

## perform fault slip analysis on each case

In [2]:
import pandas as pd
import numpy as np
from fault_slip_analysis import FSA_stress_based

n_cases = 2
n_faults = 12
fault_info = pd.read_csv('data/raw/fault_strike_dip.csv')
coor_fault = np.load('data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy')
FSA_by_fault = np.full((107,117,5,6), np.nan)
for i in range(n_cases):
    print(f"Processing case{i+1}...")
    for j in range(n_faults):
        row = fault_info.loc[fault_info['fault_id'] == j].iloc[0]
        fault_slip = FSA_stress_based(
            stress_folder_path = "data/250819_stresses",
            parameter_file_path = "data/params_responses/250819_CMG_parameters.csv",
            fault_cell_file_path = 'data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy',
            # save_folder_path = "data/250819_FSA_stress",
            case_name = f"case{i+1}", 
            fault_id = j,
            fault_strike = row['fault_strike_deg'],
            fault_dip = row['dip_angle_deg']
            )

        # save analysis to the specific fault 
        fault_id_mask = (coor_fault[:,:,:,3] == j)
        FSA_by_fault[fault_id_mask] = fault_slip[fault_id_mask]

    # save results for one case
    np.save(f'data/250819_FSA/case{i+1}_FSA.npy', FSA_by_fault)

Processing case1...
Processing case2...


## combine all cases in a numpy array (n_cases,n_faults,n_times)

In [17]:
import numpy as np
from pathlib import Path

base_path = Path('.')
FSA_folder = base_path/'data'/'250819_FSA'
save_file_prefix = '250915'

n_cases = 2; n_faults = 12; n_times = 6
fault_info = pd.read_csv('data/raw/fault_strike_dip.csv')
coor_fault = np.load('data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy')
FSA_combined = np.full((n_cases,n_faults,n_times), np.nan)
for case_num in range(1,n_cases+1):
    print(f"Processing case{case_num}...")
    FSA = np.load(FSA_folder/f'case{case_num}_FSA.npy')

    for fault_id in range(0,n_faults):
        # save analysis to the specific fault 
        fault_id_mask = (coor_fault[:,:,:,3] == fault_id)
        FSA_combined[case_num-1,fault_id,:] = np.nansum(FSA[fault_id_mask],axis=0)

# save as csv
np.save(base_path/'data'/f'{save_file_prefix}_FSA_combined.npy',FSA_combined)

# np.savetxt(base_path/'data'/f'{save_file_prefix}_FSA_combined.csv',FSA_combined,delimiter=",",fmt="%.4f")

Processing case1...
Processing case2...


In [23]:
print(FSA_combined.shape)
print(FSA_combined[0,:,:])

(2, 12, 6)
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0. 394. 394. 394. 394. 394.]
 [  0.   0.   0.   0.   0.   0.]
 [ 60. 335. 335. 335. 335. 335.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]


## calculate the (number of cases with fault slip / total cases)

In [25]:
import numpy as np
from pathlib import Path

save_file_prefix = '250915'
base_path = Path('.')

FSA_combined = np.load(base_path/'data'/f'{save_file_prefix}_FSA_combined.npy')
no_slip_count = np.sum(FSA_combined == 0, axis=0)
slip_probability = 1 - no_slip_count/FSA_combined.shape[0]
# np.savetxt(base_path/'data'/f'{save_file_prefix}_FSA_probability.csv',slip_probability,delimiter=",",fmt="%.4f")
# Save as CSV
df = pd.DataFrame(
    slip_probability,
    columns=[f"time_{t+1}" for t in range(n_times)]
)
df.insert(0, "fault_id", range(n_faults))

df.to_csv(base_path/'data'/f'{save_file_prefix}_FSA_probability.csv', index=False, float_format="%.4f")